## Extract dataset used by a paper with OpenAlex API

Exemple of request for the paper "Exploring Soothness...":  https://api.openalex.org/works/https://doi.org/10.1007/978-3-031-16443-9_4?select=referenced_works

In [28]:
import requests
import numpy as np
from pprint import pprint
#Dictionnary of papers with names as key and DOIs as value
papers = {
    "Exploring Smoothness and Class-Separation for Semi-supervised Medical Image Segmentation":"10.1007/978-3-031-16443-9_4",
    "MaxStyle: Adversarial Style Composition for Robust Medical Image Segmentation":"10.1007/978-3-031-16443-9_15",
    "Swin-Unet: Unet-like Pure Transformer for Medical Image Segmentation":"10.1007/978-3-031-25066-8_9",
    "nnFormer: Volumetric Medical Image Segmentation via a 3D Transformer":"10.48550/arXiv.2109.03201",
    "TransUNet: Transformers Make Strong Encoders for Medical Image Segmentation":"10.48550/arXiv.2102.04306"
}

#Dictionnary with dataset names as key and openalex id as value. We associate an openalex ID because it's the value in the "referenced_works" field given by the API.
#To save a request, I use directly the OpenAlex ID otherwise you need to make a request with the ID to get the DOI or the name.
datasets_id = {
    "ACDC":"https://openalex.org/W2804047627",
    "LA":"https://openalex.org/W3093394156",
    "PROMISE12":"https://openalex.org/W2106033751",
    "MSCMRSeg":"https://openalex.org/W4312016581",
    "M&Ms":"https://openalex.org/W4226199676",
    "Medical Decathlon":"https://openalex.org/W3172681723",
    "I2CVB":"https://openalex.org/W2049522781",
    "BRATS":"https://openalex.org/W1641498739"
}

datasets_context = {
    "ACDC":"Cardiac",
    "LA":"Cardiac",
    "MSCMRSeg":"Cardiac",
    "M&Ms":"Cardiac",
    "PROMISE12":"Prostate",
    "Medical Decathlon":"Prostate",
    "I2CVB":"Prostate",
    "BRATS":"Brain",
    "Synapse":"Multi-organ"
}

In [33]:
#Base url for the request
base_url = "https://api.openalex.org/works/https://doi.org/"

#To only get the field of referenced works in the response
query_param = {
    "select":"referenced_works"
}


#Will be filled with datasets found in the "referenced works" response of each paper
datasets_used = {p:[] for p in papers}

for paper_name in papers:
    request_url = base_url + papers[paper_name]
    request = requests.get(request_url, params=query_param)
    json_response = request.json()
    for ds in datasets_id:
        if datasets_id[ds] in json_response['referenced_works']:
            datasets_used[paper_name].append(ds)
pprint(datasets_used)

{'Exploring Smoothness and Class-Separation for Semi-supervised Medical Image Segmentation': ['ACDC',
                                                                                              'LA'],
 'MaxStyle: Adversarial Style Composition for Robust Medical Image Segmentation': ['PROMISE12',
                                                                                   'I2CVB'],
 'Swin-Unet: Unet-like Pure Transformer for Medical Image Segmentation': [],
 'TransUNet: Transformers Make Strong Encoders for Medical Image Segmentation': [],
 'nnFormer: Volumetric Medical Image Segmentation via a 3D Transformer': ['ACDC']}


## Extract papers that use a dataset with OpenAlex

Exemple of request for ACDC: https://api.openalex.org/works?page=2&filter=cites:W2804047627

In [30]:
datasets_used = {p:[] for p in papers}
for dataset in datasets_id:
    next_page = True
    page_number = 1
    while next_page:
        base_url = "https://api.openalex.org/works"
        query_param = {
            "filter":f"cites:{datasets_id[dataset]}",
            "page":page_number
        }
        request = requests.get(base_url,params=query_param)
        request_json = request.json()
        
        for res in request_json["results"]:
            if res["title"] in papers:
                datasets_used[res["title"]].append(dataset)
        if not request_json["results"]:
            next_page = False
        else:
            page_number += 1

pprint(datasets_used)

{'Exploring Smoothness and Class-Separation for Semi-supervised Medical Image Segmentation': ['ACDC',
                                                                                              'LA'],
 'MaxStyle: Adversarial Style Composition for Robust Medical Image Segmentation': ['PROMISE12',
                                                                                   'I2CVB'],
 'Swin-Unet: Unet-like Pure Transformer for Medical Image Segmentation': [],
 'TransUNet: Transformers Make Strong Encoders for Medical Image Segmentation': [],
 'nnFormer: Volumetric Medical Image Segmentation via a 3D Transformer': []}


## Extract Paper Task with the abstract contains in API Response

Exemple of request for the paper "Exploring Soothness...":  https://api.openalex.org/works/https://doi.org/10.1007/978-3-031-16443-9_4?select=abstract_inverted_index

In [34]:
#Base url for the request
base_url = "https://api.openalex.org/works/https://doi.org/"

#To only get the field of referenced works in the response
query_param = {
    "select":"abstract_inverted_index"
}

key_words = ["Heart","Cardiac","Prostate","Multi-organ","Brain","Liver","Aorta","Gallblader","Spleen","Kidney","Pancreas","Stomach","Lung","Breast","Chest","Skin","Eye","Retina"]
paper_theme = {p:[] for p in papers}

for paper_name in papers:
    request_url = base_url + papers[paper_name]
    request = requests.get(request_url, params=query_param)
    json_response = request.json()
    abstract_words = list(json_response["abstract_inverted_index"])
    for kw in key_words:
        if np.array([kw.upper() in w.upper() for w in abstract_words]).any():
            paper_theme[paper_name].append(kw)

paper_theme

{'Exploring Smoothness and Class-Separation for Semi-supervised Medical Image Segmentation': [],
 'MaxStyle: Adversarial Style Composition for Robust Medical Image Segmentation': ['Cardiac',
  'Prostate'],
 'Swin-Unet: Unet-like Pure Transformer for Medical Image Segmentation': ['Cardiac',
  'Multi-organ'],
 'nnFormer: Volumetric Medical Image Segmentation via a 3D Transformer': [],
 'TransUNet: Transformers Make Strong Encoders for Medical Image Segmentation': ['Cardiac',
  'Multi-organ']}

## Merge the information between context obtained with abstract and context obtained with referenced datasets 

In [35]:
print("==============================Before fusion using only abstract==============================")
pprint(paper_theme)
for paper_name in papers:
    datasets = datasets_used[paper_name]
    for ds in datasets:
        if datasets_context[ds] not in paper_theme[paper_name]:
            paper_theme[paper_name].append(datasets_context[ds])
print("\n==============================After fusion using both abstract and referenced papers==============================")
pprint(paper_theme)

==============================Before fusion using only abstract==============================
{'Exploring Smoothness and Class-Separation for Semi-supervised Medical Image Segmentation': [],
 'MaxStyle: Adversarial Style Composition for Robust Medical Image Segmentation': ['Cardiac',
                                                                                   'Prostate'],
 'Swin-Unet: Unet-like Pure Transformer for Medical Image Segmentation': ['Cardiac',
                                                                          'Multi-organ'],
 'TransUNet: Transformers Make Strong Encoders for Medical Image Segmentation': ['Cardiac',
                                                                                 'Multi-organ'],
 'nnFormer: Volumetric Medical Image Segmentation via a 3D Transformer': []}

==============================After fusion using both abstract and referenced papers==============================
{'Exploring Smoothness and Class-Separation for Semi-supervise